In [1]:
import pandas as pd
import numpy as np
import re
import os 
from datetime import datetime, timedelta

In [ ]:
def check_intervalo(timestamp1, timestamp2, timestamp3, timestamp4):
    time1 = datetime.fromtimestamp(timestamp1)
    time2 = datetime.fromtimestamp(timestamp2)
    time3 = datetime.fromtimestamp(timestamp3)
    time4 = datetime.fromtimestamp(timestamp4)

    # Verifica se a diferença entre todos os timestamps está dentro de 220 minutos
    time_diffs = [
        abs(time2 - time1),
        abs(time3 - time1),
        abs(time4 - time1),
        abs(time2 - time3),
        abs(time2 - time4),
        abs(time3 - time4)
    ]
    
    # Verifica se todos estão dentro de 220 minutos
    time_difference_ok = all(diff <= timedelta(minutes=220) for diff in time_diffs)
    
    # Verifica se time1 é o maior de todos
    time1_maior = time1 > time2 and time1 > time3 and time1 > time4

    return time_difference_ok and time1_maior

In [ ]:
links= {'100':['ap-am', 'am-ap', 'ap-pa', 'pa-ap', 'pa-to', 'to-pa', 'to-df', 'df-to', 'df-mt', 'mt-df', 'mt-ro', 'ro-mt', 'mt-ms', 'ms-mt', 'df-ma', 'ma-df', 'ma-pi', 'pi-ma', 'pi-ba', 'ba-pi', 'df-rj', 'rj-df', 'rj-es', 'es-rj', 'ba-df', 'df-ba', 'pi-ce', 'ce-pi', 'ce-rn', 'rn-ce', 'rn-pb', 'pb-rn', 'pb-pe', 'pe-pb', 'pe-al', 'al-pe', 'al-se', 'se-al', 'se-ba', 'ba-se', 'ce-ba', 'ba-ce', 'ba-mg', 'mg-ba', 'ba-es', 'es-ba', 'mg-sp', 'sp-mg', 'sp-rs', 'rs-sp', 'pr-sc', 'sc-pr', 'rs-sc', 'sc-rs'], '200':['ce-sp', 'sp-ce', 'rj-sp', 'sp-rj', 'pr-rs', 'rs-pr'], '300':['pr-sp', 'sp-pr'], '40':['pb-ba', 'ba-pb'], '20':['go-df', 'df-go', 'df-sp', 'sp-df', 'mg-rj', 'rj-mg'], '6':['ac-ro', 'ro-ac'], '3':['am-df', 'df-am'], '1':['am-rr', 'rr-am', 'rr-ce', 'ce-rr'], '10':['mt-go', 'go-mt', 'ms-go', 'go-ms', 'ms-pr', 'pr-ms', 'df-mg', 'mg-df', 'mg-es', 'es-mg', 'pa-ma', 'ma-pa', 'df-ce', 'ce-df']}

In [3]:
df = pd.read_csv('../datasets/alterados/saida/bbr/traceroute da vazao pr-ce.csv')
#1) quantidade de saltos 
def contar_ocorrencias(row):
        ocorrencias = row.astype(str).str.contains('bkb.rnp.br|No Hostname|pop').sum() #bkb.rnp.br
        return ocorrencias

#aplica a função: 
df['Hop_count'] = df.apply(contar_ocorrencias, axis=1)

#2)limpeza para encontrar os links 
def substituir_prefixo(df):
        def processar_celula(valor):
            if isinstance(valor, str):
                if '.bkb.rnp.br' in valor:
                    prefixo = valor.split('.')[0].split('-')
                    prefixo = prefixo[0] + '-' + prefixo[1]
                    # Remover números e substrings 'lan' e 'mx'
                    prefixo = re.sub(r'\d+|lan|mx', '', prefixo)
                    # Dividir a string no hífen
                    partes = prefixo.split('-')
                    # Verificar e ajustar a primeira parte
                    if len(partes) > 0 and len(partes[0]) == 3:
                        partes[0] = partes[0][1:]  # Remover o primeiro caractere da primeira parte
                    # Verificar e ajustar a segunda parte
                    if len(partes) > 1 and len(partes[1]) == 3:
                        partes[1] = partes[1][1:]  # Remover o primeiro caractere da segunda parte
                    return '-'.join(partes)
                else:
                    return np.nan
            else:
                return np.nan
        df.iloc[:, 2:-2] = df.iloc[:, 2:-2].map(processar_celula)
        return df
#aplicar a função
#df = substituir_prefixo(df)


#3) verificar o link com menor velocidade:
def obter_menor_chave(linha, links):
    chaves_encontradas = []
    links_gargalo = []
    
    for link in linha:
        for chave, valores in links.items():
            if link in valores:
                chaves_encontradas.append(int(chave))
                links_gargalo.append(link)
    
    # Retornar a menor chave e o link correspondente, ou NaN se nenhum for encontrado
    if chaves_encontradas:
        menor_chave = min(chaves_encontradas)
        indice_menor = chaves_encontradas.index(menor_chave)
        link_gargalo = links_gargalo[indice_menor]
        return menor_chave, link_gargalo
    else:
        return np.nan, np.nan
    
    
def aplicar_bottleneck(df, links):
    df[['Bottleneck', 'Link_bottleneck']] = df.apply(lambda linha: obter_menor_chave(linha, links), axis=1, result_type='expand')
    return df
#aplicar a função
#df = aplicar_bottleneck(df, links)

#4) renomear colunas
def renomear_coluna(df):
     df = df.rename(columns = {'0':'Timestamp'})
     df = df.rename(columns = {'1':'Datetime'})
     return df
#chamar a função
#df = renomear_coluna(df)

# #4) deixar no dataframe resultante apenas as colunas importantes 
def selecionar_colunas(df):
    # Seleciona as colunas desejadas
    colunas_desejadas = ['Timestamp', 'Datetime', 'Hop_count', 'Bottleneck', 'Link_bottleneck', 'Vazao']
    
    # Retorna o DataFrame apenas com essas colunas
    return df[colunas_desejadas]

#df= selecionar_colunas(df)

In [4]:
import os
def salvar_df_para_csv(df, caminho, nome_arquivo):
    if not os.path.exists(caminho):
        os.makedirs(caminho)
    caminho_completo = os.path.join(caminho, nome_arquivo)

    df.to_csv(caminho_completo, index=False)
    
    print(f"DataFrame salvo em: {caminho_completo}")


In [5]:
#fazer a parte iterativa 
pasta_raiz = '../datasets/alterados/saida'
pasta_destino = '../datasets/traceroute_vazao'

for subpasta in os.listdir(pasta_raiz):
    caminho_subpasta = os.path.join(pasta_raiz, subpasta)
    #print(caminho_subpasta)
    # Verifica se o caminho é uma pasta
    if os.path.isdir(caminho_subpasta):
        #print(f"Processando a pasta: {subpasta}")
        tcp = subpasta
        # Itera sobre os arquivos CSV dentro da subpasta
        for arquivo in os.listdir(caminho_subpasta):
            if arquivo.endswith(".csv"):
                caminho_arquivo = os.path.join(caminho_subpasta, arquivo)
                #print(f"  Lendo o arquivo: {arquivo}")
                # Lê o arquivo CSV
                df = pd.read_csv(caminho_arquivo)
                df['Hop_count'] = df.apply(contar_ocorrencias, axis=1)
                df = substituir_prefixo(df)
                df = aplicar_bottleneck(df, links)
                df = renomear_coluna(df)
                df= selecionar_colunas(df)
                nome_csv = tcp+ ' '+ arquivo
                salvar_df_para_csv(df, pasta_destino, nome_csv)




DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao ac-am.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao to-go.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao ac-ba.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao ac-ap.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao to-se.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao to-mt.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao ac-ce.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao to-es.csv
DataFrame salvo em: ../datasets/traceroute_vazao/cubic traceroute da vazao to-sp.csv
DataFrame salvo em: ../datasets/traceroute_vazao/bbr traceroute da vazao ac-am.csv
DataFrame salvo em: ../datasets/traceroute_vazao/bbr traceroute da vazao to-go.csv
DataFrame salvo em: ../datasets/traceroute_vazao/bbr traceroute da va